In [1]:
import tkinter as tk
from tkinter import messagebox, filedialog
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests
import tempfile
import os

def register_face_to_server(server_url, user_id, person_name, image_path):
    data = {'user_id': user_id, 'name': person_name}

    try:
        with open(image_path, 'rb') as image_file:
            files = {'image': image_file}
            response = requests.post(server_url, data=data, files=files)

        if response.status_code in [200, 201]:
            response_json = response.json()
            if 'message' in response_json:
                messagebox.showinfo("Cadastro", response_json['message'])
        else:
            messagebox.showerror("Erro", f"Falha ao cadastrar rosto: {response.text}")

    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao cadastrar o rosto: {str(e)}")

def detect_faces_from_webcam_and_send_to_server(server_register_url, server_recognize_url):
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    stop = False
    face_detected = False
    temp_image_path = None  
    loaded_image = None  
    frozen_face_encodings = []  
    is_image_loaded = False  # Flag para saber se uma imagem foi carregada

    def on_close():
        nonlocal stop, temp_image_path
        stop = True
        print("Fechando a janela e liberando a câmera...")
        if video_capture.isOpened():
            video_capture.release()
        if temp_image_path and os.path.exists(temp_image_path):
            try:
                os.remove(temp_image_path)  
            except PermissionError:
                print(f"Erro: O arquivo '{temp_image_path}' ainda está em uso.")
        root.destroy()

    root = tk.Tk()
    root.title("Webcam ao Vivo")

    label = tk.Label(root)
    label.pack()

    frame_controls = tk.Frame(root)
    frame_controls.pack()

    btn_register = tk.Button(frame_controls, text="Cadastrar Rosto", command=lambda: check_and_register_face(server_register_url))
    btn_register.pack(side=tk.LEFT, padx=5)
    btn_register["state"] = tk.DISABLED  

    btn_recognize = tk.Button(frame_controls, text="Reconhecer Rosto", command=lambda: recognize_face(server_recognize_url))
    btn_recognize.pack(side=tk.LEFT, padx=5)
    btn_recognize["state"] = tk.DISABLED  

    btn_resume = tk.Button(frame_controls, text="Retomar Webcam", command=lambda: resume_webcam())
    btn_resume.pack(side=tk.LEFT, padx=5)
    btn_resume["state"] = tk.DISABLED  

    btn_load_image = tk.Button(frame_controls, text="Carregar Imagem", command=lambda: load_image())
    btn_load_image.pack(side=tk.LEFT, padx=5)

    root.bind('<Control-q>', lambda event: on_close())
    root.protocol("WM_DELETE_WINDOW", on_close)

    def resume_webcam():
        nonlocal face_detected, frozen_face_encodings, is_image_loaded
        face_detected = False
        frozen_face_encodings = []
        is_image_loaded = False  # Reseta o estado de imagem carregada
        btn_register["state"] = tk.DISABLED
        btn_recognize["state"] = tk.DISABLED
        btn_resume["state"] = tk.DISABLED
        update_frame()

    def update_frame():
        nonlocal face_detected, temp_image_path, frozen_face_encodings
        if stop or face_detected or is_image_loaded:
            return

        ret, frame = video_capture.read()
        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []

        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.98:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                face_locations.append((startY, endX, endY, startX))
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        label.imgtk = imgtk
        label.configure(image=imgtk)

        if len(face_locations) > 0:
            print("Rosto detectado!")
            frozen_face_encodings = face_recognition.face_encodings(frame, face_locations)

            if len(frozen_face_encodings) > 0:
                _, temp_image_path = tempfile.mkstemp(suffix='.png')
                cv2.imwrite(temp_image_path, frame)
                os.close(_)  
                btn_register["state"] = tk.NORMAL
                btn_recognize["state"] = tk.NORMAL
                btn_resume["state"] = tk.NORMAL
            else:
                print("Falha ao gerar codificações faciais.")
        else:
            label.after(10, update_frame)

    def load_image():
        nonlocal frozen_face_encodings, temp_image_path, face_detected, loaded_image, is_image_loaded

        file_path = filedialog.askopenfilename(title="Escolha uma imagem", filetypes=[("Imagens", "*.jpg;*.jpeg;*.png")])
        if not file_path:
            return  # Se o caminho for vazio, sai da função

        face_detected = True  # Pausa a webcam
        is_image_loaded = True  # Sinaliza que uma imagem foi carregada

        try:
            # Carrega a imagem e converte para formato numpy e RGB
            image = Image.open(file_path)
            loaded_image = np.array(image)
            rgb_image = loaded_image[:, :, ::-1]  # Converte para RGB

            # Exibe a imagem carregada
            frame_image = Image.fromarray(rgb_image)
            imgtk = ImageTk.PhotoImage(image=frame_image)
            label.imgtk = imgtk
            label.configure(image=imgtk)

            # Tenta detectar rostos na imagem original
            face_locations = face_recognition.face_locations(rgb_image)
            frozen_face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

            if len(frozen_face_encodings) > 0:
                temp_image_path = file_path  # Se os rostos forem detectados, salva o caminho da imagem
                messagebox.showinfo("Sucesso", "Rosto detectado na imagem carregada.")
            else:
                messagebox.showinfo("Erro", "Nenhum rosto foi detectado na imagem.")

            # Habilita os botões após carregar a imagem, independentemente da detecção de rostos
            btn_register["state"] = tk.NORMAL
            btn_recognize["state"] = tk.NORMAL
            btn_resume["state"] = tk.NORMAL

        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro ao carregar a imagem: {str(e)}")

    def check_and_register_face(server_register_url):
        recognized = recognize_face(server_register_url)
        if not recognized:
            open_registration_form(server_register_url)

    def open_registration_form(server_register_url):
        nonlocal temp_image_path

        def on_submit():
            user_id = entry_user_id.get()
            person_name = entry_name.get()
            if user_id and person_name:
                if temp_image_path:
                    register_face_to_server(server_register_url, user_id, person_name, temp_image_path)
                else:
                    messagebox.showerror("Erro", "Nenhuma imagem de rosto capturada para cadastrar.")
                top.destroy()
            else:
                messagebox.showerror("Erro", "Por favor, preencha todos os campos.")

        top = tk.Toplevel(root)
        top.title("Cadastro de Rosto")

        tk.Label(top, text="User ID:").pack()
        entry_user_id = tk.Entry(top)
        entry_user_id.pack()

        tk.Label(top, text="Nome:").pack()
        entry_name = tk.Entry(top)
        entry_name.pack()

        tk.Button(top, text="Cadastrar", command=on_submit).pack()

    def recognize_face(server_recognize_url):
        nonlocal frozen_face_encodings, loaded_image, is_image_loaded

        # Verifica se há codificações faciais congeladas
        if len(frozen_face_encodings) == 0:
            messagebox.showinfo("Reconhecimento", "Nenhum rosto detectado ou codificação facial disponível.")
            return False

        data_to_send = {
            'encodings': [encoding.tolist() for encoding in frozen_face_encodings]
        }

        try:
            response = requests.post(server_recognize_url, json=data_to_send)
            if response.status_code == 200:
                response_data = response.json()

                if "name" in response_data and "user_id" in response_data:
                    name = response_data.get("name", "Nome desconhecido")
                    user_id = response_data.get("user_id", "ID desconhecido")
                    messagebox.showinfo("Reconhecimento", f"Nome: {name}\nUser ID: {user_id}")
                    return True
                else:
                    messagebox.showinfo("Reconhecimento", "Nenhum rosto correspondente encontrado.")
            else:
                messagebox.showerror("Erro", f"Erro ao reconhecer o rosto. Código: {response.status_code}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao comunicar com o servidor: {e}")
    
        return False

    update_frame()
    root.mainloop()

server_register_url = 'http://127.0.0.1:5000/register'
server_recognize_url = 'http://127.0.0.1:5000/recognize'
detect_faces_from_webcam_and_send_to_server(server_register_url, server_recognize_url)


Rosto detectado!


: 

In [4]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests
import tempfile
import os

def register_face_to_server(server_url, user_id, person_name, image_path):
    """
    Envia a imagem, user_id e o nome para o servidor usando multipart/form-data.
    """
    data = {'user_id': user_id, 'name': person_name}

    try:
        with open(image_path, 'rb') as image_file:
            files = {'image': image_file}
            response = requests.post(server_url, data=data, files=files)

        if response.status_code in [200, 201]:
            response_json = response.json()
            if 'message' in response_json:
                messagebox.showinfo("Cadastro", response_json['message'])
        else:
            messagebox.showerror("Erro", f"Falha ao cadastrar rosto: {response.text}")

    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao cadastrar o rosto: {str(e)}")

def detect_faces_from_webcam_and_send_to_server(server_register_url, server_recognize_url):
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    stop = False
    face_detected = False
    temp_image_path = None  # Para armazenar a imagem temporária capturada
    frozen_frame = None  # Frame congelado após a detecção
    frozen_face_locations = []  # Localizações do rosto congeladas
    frozen_face_encodings = []  # Codificações faciais congeladas

    def on_close():
        nonlocal stop, temp_image_path
        stop = True
        print("Fechando a janela e liberando a câmera...")
        if video_capture.isOpened():
            video_capture.release()
        if temp_image_path and os.path.exists(temp_image_path):
            try:
                os.remove(temp_image_path)  # Remover o arquivo temporário
            except PermissionError:
                print(f"Erro: O arquivo '{temp_image_path}' ainda está em uso.")
        root.destroy()

    root = tk.Tk()
    root.title("Webcam ao Vivo")

    label = tk.Label(root)
    label.pack()

    frame_controls = tk.Frame(root)
    frame_controls.pack()

    # Modificado para chamar `check_and_register_face`
    btn_register = tk.Button(frame_controls, text="Cadastrar Rosto", command=lambda: check_and_register_face(server_register_url, server_recognize_url))
    btn_register.pack(side=tk.LEFT, padx=5)
    btn_register["state"] = tk.DISABLED  # Desabilitar até que um rosto seja detectado

    btn_recognize = tk.Button(frame_controls, text="Reconhecer Rosto", command=lambda: recognize_face(server_recognize_url))
    btn_recognize.pack(side=tk.LEFT, padx=5)
    btn_recognize["state"] = tk.DISABLED  # Desabilitar até que um rosto seja detectado

    btn_resume = tk.Button(frame_controls, text="Retomar Webcam", command=lambda: resume_webcam())
    btn_resume.pack(side=tk.LEFT, padx=5)
    btn_resume["state"] = tk.DISABLED  # Desabilitar até que o feed seja pausado

    root.bind('<Control-q>', lambda event: on_close())
    root.protocol("WM_DELETE_WINDOW", on_close)

    def resume_webcam():
        nonlocal face_detected, frozen_frame, frozen_face_locations, frozen_face_encodings
        face_detected = False
        frozen_frame = None
        frozen_face_locations = []
        frozen_face_encodings = []
        btn_register["state"] = tk.DISABLED
        btn_recognize["state"] = tk.DISABLED
        btn_resume["state"] = tk.DISABLED
        update_frame()

    def update_frame():
        nonlocal face_detected, temp_image_path, frozen_frame, frozen_face_locations, frozen_face_encodings
        if stop or face_detected:
            return

        ret, frame = video_capture.read()
        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]

        # Pré-processar a imagem para a detecção de rosto
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []

        # Percorrer as detecções
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            # Filtrar detecções fracas (threshold de confiança pode ser ajustado)
            if confidence > 0.5:
                # Obter as coordenadas da caixa delimitadora para o rosto detectado
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # Adicionar à lista de localizações de rosto
                face_locations.append((startY, endX, endY, startX))

                # Desenhar a caixa delimitadora ao redor do rosto
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # Converter o frame para RGB (compatível com tkinter)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        # Atualizar o label tkinter com a nova imagem
        label.imgtk = imgtk
        label.configure(image=imgtk)

        # Verificar se algum rosto foi detectado
        if len(face_locations) > 0:
            print("Rosto detectado!")

            # Obter as codificações faciais para os rostos detectados
            frozen_face_encodings = face_recognition.face_encodings(frame, face_locations)

            if len(frozen_face_encodings) > 0:
                # Salvar a imagem congelada
                _, temp_image_path = tempfile.mkstemp(suffix='.png')
                cv2.imwrite(temp_image_path, frame)
                os.close(_)  # Fechar o arquivo corretamente

                btn_register["state"] = tk.NORMAL
                btn_recognize["state"] = tk.NORMAL
                btn_resume["state"] = tk.NORMAL
            else:
                print("Falha ao gerar codificações faciais.")

        else:
            label.after(10, update_frame)

    def check_and_register_face(server_register_url, server_recognize_url):
        """
        Tenta reconhecer a face antes de permitir o cadastro.
        Se a face for reconhecida (status 200), exibe a mensagem.
        Se a face não for reconhecida, procede com o cadastro.
        """
        recognized = recognize_face(server_recognize_url)
        if not recognized:
            # Se o rosto não for reconhecido, permitir o cadastro
            open_registration_form(server_register_url)

    def open_registration_form(server_register_url):
        nonlocal temp_image_path

        def on_submit():
            user_id = entry_user_id.get()
            person_name = entry_name.get()
            if user_id and person_name:
                if temp_image_path:
                    register_face_to_server(server_register_url, user_id, person_name, temp_image_path)
                else:
                    messagebox.showerror("Erro", "Nenhuma imagem de rosto capturada para cadastrar.")
                top.destroy()
            else:
                messagebox.showerror("Erro", "Por favor, preencha todos os campos.")

        top = tk.Toplevel(root)
        top.title("Cadastro de Rosto")

        tk.Label(top, text="User ID:").pack()
        entry_user_id = tk.Entry(top)
        entry_user_id.pack()

        tk.Label(top, text="Nome:").pack()
        entry_name = tk.Entry(top)
        entry_name.pack()

        tk.Button(top, text="Cadastrar", command=on_submit).pack()

    def recognize_face(server_recognize_url):
        nonlocal frozen_face_encodings
        if len(frozen_face_encodings) == 0:
            messagebox.showinfo("Reconhecimento", "Nenhum rosto detectado ou imagem congelada.")
            return False

        # Preparar os dados para enviar ao servidor
        data_to_send = {
            'encodings': [encoding.tolist() for encoding in frozen_face_encodings]  # Converter arrays numpy para listas
        }

        # Enviar as codificações faciais para o servidor
        try:
            response = requests.post(server_recognize_url, json=data_to_send)
            print(f"Status do servidor: {response.status_code}")  # Debug do status da resposta

            if response.status_code == 200:
                response_data = response.json()
                print("Server response data:", response_data)  # Debug: Print the response data
                
                if "name" in response_data and "user_id" in response_data:
                    # Obter o nome e user_id da resposta do servidor
                    name = response_data.get("name", "Nome desconhecido")
                    user_id = response_data.get("user_id", "ID desconhecido")
                    
                    # Exibir os dados no messagebox
                    messagebox.showinfo("Reconhecimento", f"Nome: {name}\nUser ID: {user_id}")
                    return True  # Retorna True se o rosto foi reconhecido
                else:
                    messagebox.showinfo("Reconhecimento", "Nenhum rosto correspondente encontrado.")
            else:
                return False  # Rosto não foi reconhecido
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao comunicar com o servidor: {e}")
        
        return False  # Retorna False se o rosto não foi reconhecido

    update_frame()
    root.mainloop()

# Testar a função com as URLs de cadastro e reconhecimento
server_register_url = 'http://127.0.0.1:5000/register'
server_recognize_url = 'http://127.0.0.1:5000/recognize'
detect_faces_from_webcam_and_send_to_server(server_register_url, server_recognize_url)


TclError: image "pyimage4" doesn't exist